<a href="https://colab.research.google.com/github/Ciss48/Bank_Transaction_Report/blob/main/Transaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load file

In [1]:
from google.colab import files
import pandas as pd

# Upload file
uploaded = files.upload()

# Đọc file
customer_df = pd.read_csv("customer_raw.csv")
transaction_df = pd.read_csv("transaction_raw.csv")

Saving customer_raw.csv to customer_raw.csv
Saving transaction_raw.csv to transaction_raw.csv


# EDA + data clean

## customer_df

### Xử lý các cột date

In [2]:
customer_df

,STT,Số CIF,Tên khách hàng,Giới tính,Ngày sinh,Đơn vị đăng ký,Ngày đăng ký,Ngày kích hoạt lần đầu,Kênh đăng ký,Ngày đăng nhập cuối cùng,Trạng thái
0,1,200000117262,NGUYEN THI HUONG,Nữ,7/20/1987,CN Hai Bà Trưng,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt
1,2,300200063101,TRAN DUY NHAT,Nam,2/19/2004,PGD số 2_CN Hà Tây,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt
2,3,3400100094172,DINH THI HIEN,Nữ,1/17/1996,PGD số 1_CN Thái Bình,01/04/2024,NaN,eKYC,NaN,Chờ kích hoạt
3,4,2700000143141,NGUYEN THI TAM,Nữ,3/24/1976,CN Bắc Ninh,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt
4,5,7900000043161,DANH DUNG,Nam,3/9/1993,CN TPHCM,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt
...,...,...,...,...,...,...,...,...,...,...,...
284982,284983,3300100015262,NGUYEN DUC TRANG,Nam,07/08/1956,PGD Phố Nối_CN Hưng Yên,7/4/2023,7/4/2023,Tại quầy,01/04/2024,Kích hoạt
284983,284984,4000000153122,VO THI HOA,Nữ,27/11/1973,CN Nghệ An,14/02/2021,2/14/2021,eKYC,01/04/2024,Kích hoạt
284984,284985,4000000137092,TRAN KIM HOAN,Nam,24/12/1961,CN Nghệ An,11/28/2023,11/28/2023,Tại quầy,01/04/2024,Kích hoạt
284985,284986,4000000153123,PHAM VAN DONG,Nam,14/06/1989,CN Nghệ An,01/02/2021,12/15/2021,eKYC,01/04/2024,Kích hoạt


In [11]:
# Đầu tiên khi nhìn qua có thể thấy dữ format chung cho các cột ngày đang là mm/dd/yyyy, để chắc chắn cần kiểm tra kĩ cái này

In [7]:
import pandas as pd
from datetime import datetime

# Hàm kiểm tra có đúng format mm/dd/yy hoặc mm/dd/yyyy không
def check_date_format(date_str):
    formats = ["%m/%d/%y", "%m/%d/%Y"]
    for fmt in formats:
        try:
            datetime.strptime(date_str, fmt)
            return True
        except ValueError:
            continue
    return False

# Danh sách các cột cần kiểm tra
date_cols = ["Ngày sinh", "Ngày đăng ký", "Ngày kích hoạt lần đầu", "Ngày đăng nhập cuối cùng"]

# kiểm tra từng cột
for col in date_cols:
    if col in customer_df.columns:
        customer_df[f"{col}_is_mmddyy"] = customer_df[col].astype(str).apply(check_date_format)
        true_count = customer_df[f"{col}_is_mmddyy"].sum()
        false_count = (~customer_df[f"{col}_is_mmddyy"]).sum()
        print(f"\nCột: {col}")
        print("Số dòng True :", true_count)
        print("Số dòng False:", false_count)



Cột: Ngày sinh
Số dòng True : 284269
Số dòng False: 718

Cột: Ngày đăng ký
Số dòng True : 95552
Số dòng False: 189435

Cột: Ngày kích hoạt lần đầu
Số dòng True : 80220
Số dòng False: 204767

Cột: Ngày đăng nhập cuối cùng
Số dòng True : 147993
Số dòng False: 136994


In [8]:
# Dữ liệu đang có dấu hiệu sai format các cột ngày rất nhiều, thử in ra để kiểm tra lý do tại sao lại sai ?

In [10]:
# Lọc các dòng False ở cột "Ngày đăng ký"
false_rows = customer_df[customer_df["Ngày đăng ký_is_mmddyy"] == False]

print(false_rows[["Ngày đăng ký"]].sample(20))  # in thử 20 dòng ngẫu nhiên
print("Tổng số dòng False:", len(false_rows))


       Ngày đăng ký
138554   27/09/2022
122182   16/11/2022
180062   23/06/2022
9785     23/02/2024
257007   24/03/2022
226823   17/05/2022
230732   13/05/2022
39936    23/10/2023
237629   29/04/2022
214369   26/05/2022
120709   22/11/2022
8517     28/02/2024
141697   21/09/2022
130542   20/10/2022
138195   27/09/2022
183726   21/06/2022
143059   19/09/2022
273385   24/02/2022
280117   20/01/2022
193599   13/06/2022
Tổng số dòng False: 189435


In [13]:
# Có thể phát hiện ra rằng 1 vài dữ liệu đang có định dạng dd/mm/yyyy
# => Cần tạo ra 1 hàm để khi gặp các dòng có format này, chuyển về format đúng
# Nguyên tắc code hoạt động sẽ là:
# Parse toàn bộ các dòng trong 4 cột date thành dạng mm/dd/yyyy, thử parse với cả 4 format phổ biến (mm/dd/yyyy, dd/mm/yyyy, và dạng 2 số năm
# Sau khi parse xong, xuất ra thống nhất dạng mm/dd/yyyy
# Các giá trị không hợp lệ giữ nguyên để tìm hiểu tiếp nguyên nhân

In [16]:
import pandas as pd
from datetime import datetime

# Hàm chuẩn hóa ngày
def normalize_date(date_str):
    if pd.isna(date_str):
        return date_str  # giữ nguyên NaN
    date_str = str(date_str).strip()
    for fmt in ("%m/%d/%Y", "%d/%m/%Y", "%m/%d/%y", "%d/%m/%y"):
        try:
            dt = datetime.strptime(date_str, fmt)
            # Xuất ra định dạng mm/dd/yyyy
            return dt.strftime("%m/%d/%Y")
        except ValueError:
            continue
    return date_str

# Các cột ngày cần xử lý
date_cols = ["Ngày sinh", "Ngày đăng ký", "Ngày kích hoạt lần đầu", "Ngày đăng nhập cuối cùng"]

# Áp dụng chuẩn hóa
for col in date_cols:
    if col in customer_df.columns:
        customer_df[col] = customer_df[col].apply(normalize_date)

# Kiểm tra kết quả
customer_df[date_cols]


,Ngày sinh,Ngày đăng ký,Ngày kích hoạt lần đầu,Ngày đăng nhập cuối cùng
0,07/20/1987,01/04/2024,01/04/2024,01/04/2024
1,02/19/2004,01/04/2024,01/04/2024,01/04/2024
2,01/17/1996,01/04/2024,NaN,NaN
3,03/24/1976,01/04/2024,01/04/2024,01/04/2024
4,03/09/1993,01/04/2024,01/04/2024,01/04/2024
...,...,...,...,...
284982,07/08/1956,07/04/2023,07/04/2023,01/04/2024
284983,11/27/1973,02/14/2021,02/14/2021,01/04/2024
284984,12/24/1961,11/28/2023,11/28/2023,01/04/2024
284985,06/14/1989,01/02/2021,12/15/2021,01/04/2024


In [17]:
# Chạy lại code
import pandas as pd
from datetime import datetime

# Hàm kiểm tra có đúng format mm/dd/yy hoặc mm/dd/yyyy không
def check_date_format(date_str):
    formats = ["%m/%d/%y", "%m/%d/%Y"]
    for fmt in formats:
        try:
            datetime.strptime(date_str, fmt)
            return True
        except ValueError:
            continue
    return False

# Danh sách các cột cần kiểm tra
date_cols = ["Ngày sinh", "Ngày đăng ký", "Ngày kích hoạt lần đầu", "Ngày đăng nhập cuối cùng"]

# kiểm tra từng cột
for col in date_cols:
    if col in customer_df.columns:
        customer_df[f"{col}_is_mmddyy"] = customer_df[col].astype(str).apply(check_date_format)
        true_count = customer_df[f"{col}_is_mmddyy"].sum()
        false_count = (~customer_df[f"{col}_is_mmddyy"]).sum()
        print(f"\nCột: {col}")
        print("Số dòng True :", true_count)
        print("Số dòng False:", false_count)



Cột: Ngày sinh
Số dòng True : 284987
Số dòng False: 0

Cột: Ngày đăng ký
Số dòng True : 284987
Số dòng False: 0

Cột: Ngày kích hoạt lần đầu
Số dòng True : 238781
Số dòng False: 46206

Cột: Ngày đăng nhập cuối cùng
Số dòng True : 241064
Số dòng False: 43923


In [18]:
# Đã xử lý thành công 2 cột Ngày sinh và Ngày đăng ký
# 2 cột  Ngày kích hoạt lần đầu và Ngày đăng nhập cuối cùng vẫn còn vấn đề, cùng kiểm tra

In [21]:
# Lọc các dòng False ở cột "Ngày đăng ký"
false_rows = customer_df[customer_df["Ngày kích hoạt lần đầu_is_mmddyy"] == False]

print(false_rows[["Ngày kích hoạt lần đầu"]])  # in thử 20 dòng ngẫu nhiên
print("Tổng số dòng False:", len(false_rows))

       Ngày kích hoạt lần đầu
2                         NaN
7                         NaN
8                         NaN
11                        NaN
12                        NaN
...                       ...
281897                    NaN
281984                    NaN
282013                    NaN
282063                    NaN
282430                    NaN

[46206 rows x 1 columns]
Tổng số dòng False: 46206


In [22]:
# Các cột này đơn giản là bị NaN, có thể là do những user có trạng thái là chưa kích hoạt thì sẽ bị NaN những cột này, cùng kiểm tra:
customer_df[customer_df['Trạng thái'] == 'Chờ kích hoạt']

,STT,Số CIF,Tên khách hàng,Giới tính,Ngày sinh,Đơn vị đăng ký,Ngày đăng ký,Ngày kích hoạt lần đầu,Kênh đăng ký,Ngày đăng nhập cuối cùng,Trạng thái,is_mmddyy,Ngày sinh_is_mmddyy,Ngày đăng ký_is_mmddyy,Ngày kích hoạt lần đầu_is_mmddyy,Ngày đăng nhập cuối cùng_is_mmddyy
2,3,3400100094172,DINH THI HIEN,Nữ,01/17/1996,PGD số 1_CN Thái Bình,01/04/2024,NaN,eKYC,NaN,Chờ kích hoạt,True,True,True,False,False
7,8,3100000060341,HA DO MINH NGOC,Nữ,09/28/1966,CN Hải Phòng,01/04/2024,NaN,Tại quầy,NaN,Chờ kích hoạt,True,True,True,False,False
8,9,5200000400158,NGUYEN THI TUYET,Nữ,04/04/1981,CN Bình Định,01/04/2024,NaN,Tại quầy,NaN,Chờ kích hoạt,True,True,True,False,False
11,12,5200000400157,LUONG THI BICH ANH,Nữ,07/23/1990,CN Bình Định,01/04/2024,NaN,Tại quầy,NaN,Chờ kích hoạt,True,True,True,False,False
12,13,5200000400156,NGUYEN THI NGOC TUYEN,Nữ,05/10/2000,CN Bình Định,01/04/2024,NaN,Tại quầy,NaN,Chờ kích hoạt,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281897,281898,4200000033884,NGUYEN THI LUONG,Nữ,07/14/1993,CN Hà Tĩnh,11/01/2022,NaN,Tại quầy,NaN,Chờ kích hoạt,True,True,True,False,False
281984,281985,2500300049892,DUONG THI THEU,Nữ,11/01/1966,PGD số 3_CN Phú Thọ,11/01/2022,NaN,Tại quầy,05/27/2022,Chờ kích hoạt,True,True,True,False,True
282013,282014,4000000091554,NGUYEN VAN NHU,Nam,05/29/1974,CN Nghệ An,11/01/2022,NaN,Tại quầy,NaN,Chờ kích hoạt,True,True,True,False,False
282063,282064,8400000042001,LY THI NGOC,Nữ,07/24/1968,CN Trà Vinh,10/01/2022,NaN,Tại quầy,NaN,Chờ kích hoạt,True,True,True,False,False


In [ ]:
# Đúng như ta suy suy nghĩ, số rows cũng = số rows false =? Sử lý xong cột date, bỏ các cột thừa đi:
customer_df = customer_df.drop(columns=["Ngày sinh_is_mmddyy", "Ngày đăng ký_is_mmddyy", "Ngày kích hoạt lần đầu_is_mmddyy", "Ngày đăng nhập cuối cùng_is_mmddyy","is_mmddyy"])

In [28]:
customer_df

,STT,Số CIF,Tên khách hàng,Giới tính,Ngày sinh,Đơn vị đăng ký,Ngày đăng ký,Ngày kích hoạt lần đầu,Kênh đăng ký,Ngày đăng nhập cuối cùng,Trạng thái
0,1,200000117262,NGUYEN THI HUONG,Nữ,07/20/1987,CN Hai Bà Trưng,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt
1,2,300200063101,TRAN DUY NHAT,Nam,02/19/2004,PGD số 2_CN Hà Tây,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt
2,3,3400100094172,DINH THI HIEN,Nữ,01/17/1996,PGD số 1_CN Thái Bình,01/04/2024,NaN,eKYC,NaN,Chờ kích hoạt
3,4,2700000143141,NGUYEN THI TAM,Nữ,03/24/1976,CN Bắc Ninh,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt
4,5,7900000043161,DANH DUNG,Nam,03/09/1993,CN TPHCM,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt
...,...,...,...,...,...,...,...,...,...,...,...
284982,284983,3300100015262,NGUYEN DUC TRANG,Nam,07/08/1956,PGD Phố Nối_CN Hưng Yên,07/04/2023,07/04/2023,Tại quầy,01/04/2024,Kích hoạt
284983,284984,4000000153122,VO THI HOA,Nữ,11/27/1973,CN Nghệ An,02/14/2021,02/14/2021,eKYC,01/04/2024,Kích hoạt
284984,284985,4000000137092,TRAN KIM HOAN,Nam,12/24/1961,CN Nghệ An,11/28/2023,11/28/2023,Tại quầy,01/04/2024,Kích hoạt
284985,284986,4000000153123,PHAM VAN DONG,Nam,06/14/1989,CN Nghệ An,01/02/2021,12/15/2021,eKYC,01/04/2024,Kích hoạt


### Thêm các cột phục vụ báo cáo

In [31]:
# Đầu tiên đối với Đơn vị đăng ký, khi đưa vào báo cáo ta không thể report theo từng đơn vị (vì quá nhiều), cũng không thể tạo 1 bộ lọc filter được
# Từ đó ta sẽ tạo ra 1 cột từ cột này, ta có thể để ý 2 chữ cuối cùng trên từng dòng sẽ biểu thị cho thành phố, từ đó có thể tạo ra được cột thành phố đăng ký
# Trước tiên sẽ kiểm tra cấu trúc của từng category

customer_df['Đơn vị đăng ký'].unique()

array(['CN Hai Bà Trưng', 'PGD số 2_CN Hà Tây', 'PGD số 1_CN Thái Bình',
       'CN Bắc Ninh', 'CN TPHCM', 'CN Sở giao dịch', 'CN Thái Bình',
       'CN Hải Phòng', 'CN Bình Định',
       'PGD Hoàng Đạo Thúy_CN Sở giao dịch', 'CN Gia Lai', 'CN Phú Thọ',
       'PGD số 3_CN Nghệ An', 'CN Lâm Đồng', 'PGD Phố Nối_CN Hưng Yên',
       'PGD Đập Đá_CN Bình Định', 'PGD Tứ kỳ_CN Hải Dương',
       'CN Ninh Bình', 'PGD Cà Mau_CN Sóc Trăng',
       'PGD Minh Khai_CN Nghệ An', 'PGD số 2_CN Thanh Hóa', 'CN Hà Tĩnh',
       'CN Hà Tây', 'CN Hà Nam', 'PGD Lệ Thủy_CN Quảng Bình',
       'PGD số 2_CN Bình Thuận', 'CN An Giang', 'CN Hưng Yên',
       'CN Đồng Nai', 'PGD Vũ Thư_CN Thái Bình', 'CN Quảng Bình',
       'CN Nghệ An', 'PGD Đông Hưng_CN Thái Bình', 'PGD so 2_CN Nghệ An',
       'PGD số 1_CN Bình Thuận', 'CN Nam Định', 'CN Thanh Hóa',
       'CN Tây Ninh', 'PGD số 4_CN Thanh Hóa',
       'PGD Quảng Trị_CN Quảng Bình', 'CN Hải Dương',
       'PGD Bố Trạch_CN Quảng Bình', 'PGD

In [ ]:
# Đa phần các từ đều theo format chung là lấy được 2 từ cuối
# Liệt kê 1 vài từ để xử lý đặc biệt
# Hai Bà Trưng -> Hà Nội, ...TPHCM -> Hồ Chí Minh, ..... Sở giao dịch -> Sở giao dịch
# ... Daklak -> Daklak, HBT -> Hà Nội,

In [32]:
import pandas as pd

def extract_city(text):
    if pd.isna(text):
        return None
    text = str(text)

    # Các trường hợp đặc biệt
    if "Hai Bà Trưng" in text:
        return "Hà Nội"
    if "TPHCM" in text or "Tp.HCM" in text or "TP.HCM" in text:
        return "Hồ Chí Minh"
    if "Sở giao dịch" in text:
        return "Sở giao dịch"
    if "Daklak" in text:
        return "Daklak"
    if "HBT" in text:
        return "Hà Nội"
    if "Hội sở" in text:
        return "Hà Nội"

    # Mặc định: lấy 2 từ cuối
    parts = text.strip().split()
    return " ".join(parts[-2:]) if len(parts) >= 2 else text

# Tạo cột mới
customer_df["Thành phố đăng ký"] = customer_df["Đơn vị đăng ký"].apply(extract_city)


customer_df["Thành phố đăng ký"].unique()

array(['Hà Nội', 'Hà Tây', 'Thái Bình', 'Bắc Ninh', 'Hồ Chí Minh',
       'Sở giao dịch', 'Thái Bình', 'Hải Phòng', 'Bình Định',
       'Gia Lai', 'Phú Thọ', 'Nghệ An', 'Lâm Đồng', 'Hưng Yên',
       'Bình Định', 'Hải Dương', 'Ninh Bình', 'Sóc Trăng', 'Thanh Hóa',
       'Hà Tĩnh', 'Hà Nam', 'Quảng Bình', 'Bình Thuận', 'An Giang',
       'Đồng Nai', 'Nam Định', 'Tây Ninh', 'Hải Dương', 'Trà Vinh',
       'Daklak', 'Kiên Giang', 'Bắc Giang', 'Long An', 'Vĩnh Phúc',
       'Bắc Ninh', 'Yên Bái', 'Lâm Đồng', 'Ninh Bình', 'Vĩnh Phúc',
       'Nam Định'], dtype=object)

In [33]:
# Thành công

In [34]:
# Thêm cột độ tuổi
import pandas as pd
from datetime import datetime


# Hàm lấy năm sinh
def get_birth_year(date_str):
    if pd.isna(date_str):
        return None
    for fmt in ("%m/%d/%Y", "%d/%m/%Y", "%Y-%m-%d", "%m/%d/%y", "%d/%m/%y"):
        try:
            return datetime.strptime(str(date_str), fmt).year
        except ValueError:
            continue
    return None  # nếu không parse được

# Tạo cột năm sinh
customer_df["Năm sinh"] = customer_df["Ngày sinh"].apply(get_birth_year)

# Tạo cột Độ tuổi
customer_df["Độ tuổi"] = customer_df["Năm sinh"].apply(lambda y: 2025 - y if y else None)

customer_df


,STT,Số CIF,Tên khách hàng,Giới tính,Ngày sinh,Đơn vị đăng ký,Ngày đăng ký,Ngày kích hoạt lần đầu,Kênh đăng ký,Ngày đăng nhập cuối cùng,Trạng thái,Thành phố đăng ký,Năm sinh,Độ tuổi
0,1,200000117262,NGUYEN THI HUONG,Nữ,07/20/1987,CN Hai Bà Trưng,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt,Hà Nội,1987,38
1,2,300200063101,TRAN DUY NHAT,Nam,02/19/2004,PGD số 2_CN Hà Tây,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt,Hà Tây,2004,21
2,3,3400100094172,DINH THI HIEN,Nữ,01/17/1996,PGD số 1_CN Thái Bình,01/04/2024,NaN,eKYC,NaN,Chờ kích hoạt,Thái Bình,1996,29
3,4,2700000143141,NGUYEN THI TAM,Nữ,03/24/1976,CN Bắc Ninh,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt,Bắc Ninh,1976,49
4,5,7900000043161,DANH DUNG,Nam,03/09/1993,CN TPHCM,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt,Hồ Chí Minh,1993,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284982,284983,3300100015262,NGUYEN DUC TRANG,Nam,07/08/1956,PGD Phố Nối_CN Hưng Yên,07/04/2023,07/04/2023,Tại quầy,01/04/2024,Kích hoạt,Hưng Yên,1956,69
284983,284984,4000000153122,VO THI HOA,Nữ,11/27/1973,CN Nghệ An,02/14/2021,02/14/2021,eKYC,01/04/2024,Kích hoạt,Nghệ An,1973,52
284984,284985,4000000137092,TRAN KIM HOAN,Nam,12/24/1961,CN Nghệ An,11/28/2023,11/28/2023,Tại quầy,01/04/2024,Kích hoạt,Nghệ An,1961,64
284985,284986,4000000153123,PHAM VAN DONG,Nam,06/14/1989,CN Nghệ An,01/02/2021,12/15/2021,eKYC,01/04/2024,Kích hoạt,Nghệ An,1989,36


In [36]:
# Tạo cột Nhóm tuổi
bins = [20, 30, 40, 50, 60, 70, 120]  # khoảng tuổi
labels = ["20-30", "31-40", "41-50", "51-60", "61-70", "71+"]  # nhãn nhóm
customer_df["Nhóm tuổi"] = pd.cut(customer_df["Độ tuổi"], bins=bins, labels=labels, right=True, include_lowest=True)

customer_df

,STT,Số CIF,Tên khách hàng,Giới tính,Ngày sinh,Đơn vị đăng ký,Ngày đăng ký,Ngày kích hoạt lần đầu,Kênh đăng ký,Ngày đăng nhập cuối cùng,Trạng thái,Thành phố đăng ký,Năm sinh,Độ tuổi,Nhóm tuổi
0,1,200000117262,NGUYEN THI HUONG,Nữ,07/20/1987,CN Hai Bà Trưng,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt,Hà Nội,1987,38,31-40
1,2,300200063101,TRAN DUY NHAT,Nam,02/19/2004,PGD số 2_CN Hà Tây,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt,Hà Tây,2004,21,20-30
2,3,3400100094172,DINH THI HIEN,Nữ,01/17/1996,PGD số 1_CN Thái Bình,01/04/2024,NaN,eKYC,NaN,Chờ kích hoạt,Thái Bình,1996,29,20-30
3,4,2700000143141,NGUYEN THI TAM,Nữ,03/24/1976,CN Bắc Ninh,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt,Bắc Ninh,1976,49,41-50
4,5,7900000043161,DANH DUNG,Nam,03/09/1993,CN TPHCM,01/04/2024,01/04/2024,eKYC,01/04/2024,Kích hoạt,Hồ Chí Minh,1993,32,31-40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284982,284983,3300100015262,NGUYEN DUC TRANG,Nam,07/08/1956,PGD Phố Nối_CN Hưng Yên,07/04/2023,07/04/2023,Tại quầy,01/04/2024,Kích hoạt,Hưng Yên,1956,69,61-70
284983,284984,4000000153122,VO THI HOA,Nữ,11/27/1973,CN Nghệ An,02/14/2021,02/14/2021,eKYC,01/04/2024,Kích hoạt,Nghệ An,1973,52,51-60
284984,284985,4000000137092,TRAN KIM HOAN,Nam,12/24/1961,CN Nghệ An,11/28/2023,11/28/2023,Tại quầy,01/04/2024,Kích hoạt,Nghệ An,1961,64,61-70
284985,284986,4000000153123,PHAM VAN DONG,Nam,06/14/1989,CN Nghệ An,01/02/2021,12/15/2021,eKYC,01/04/2024,Kích hoạt,Nghệ An,1989,36,31-40


## Transaction_df

In [37]:
transaction_df

,Số CIF,Tên khách hàng,Dịch vụ,Tổng số tiền giao dịch,Đơn vị tiền tệ,Mã lỗi,Mô tả mã lỗi,Mã giao dịch,Thời gian giao dịch,Trạng thái
0,4000000118280,NGUYEN ANH TUAN,Chuyển tiền nhanh 247 qua tài khoản,2620000,VND,0,Hạch toán Thành công,00000FT542403206051,01/02/2024 23:38:53,Giao dịch thành công
1,3600200011641,BUI THI HUONG,Chuyển tiền nhanh 247 qua tài khoản,1490000,VND,0,Hạch toán Thành công,00000FT542403206052,01/02/2024 23:38:51,Giao dịch thành công
2,6000200061082,NGUYEN THI HONG HAU,Chuyển tiền nhanh 247 qua tài khoản,12000000,VND,98,Hạch toán Timeout,00000FT542403206049,01/02/2024 23:38:28,Giao dịch nghi vấn
3,7200000014564,TRAN THANH LOC,Chuyển tiền nhanh 247 qua tài khoản,100000,VND,0,Hạch toán Thành công,00000FT542403206047,01/02/2024 23:38:08,Giao dịch thành công
4,3100000041542,DANG THI LUYEN,Hóa đơn tiền điện,538428,VND,0,Gạch nợ Thành công,00000FT602403293681,01/02/2024 23:38:07,Giao dịch thành công
...,...,...,...,...,...,...,...,...,...,...
755013,6000000141882,NGUYEN HONG PHONG,Chuyển tiền nhanh 247 qua tài khoản,1000000,VND,0,Hạch toán Thành công,00000FT542406201987,02/03/2024 00:07:07,Giao dịch thành công
755014,6000000134751,DAU THI THAO,Chuyển tiền nhanh 247 qua tài khoản,18000000,VND,0,Hạch toán Thành công,00000FT542406201971,02/03/2024 00:06:43,Giao dịch thành công
755015,8900000034641,NGUYEN THI TUYET TRINH,Hóa đơn tiền điện,718430,VND,0,Gạch nợ Thành công,00000FT602406200522,02/03/2024 00:05:36,Giao dịch thành công
755016,8900000034641,NGUYEN THI TUYET TRINH,Hóa đơn tiền điện,662431,VND,0,Gạch nợ Thành công,00000FT602406200521,02/03/2024 00:05:00,Giao dịch thành công


In [38]:
# Đầu tiên ta không biết được liệu thời gian giao dịch có đúng format không, để tận dụng code cũ ta sẽ tách nó phần ngày với phần thời gian riêng

In [39]:
# Parse datetime
transaction_df["Thời gian giao dịch"] = pd.to_datetime(transaction_df["Thời gian giao dịch"], format="%d/%m/%Y %H:%M:%S")

# Tách ra cột mới time (giờ phút giây)
transaction_df["time"] = transaction_df["Thời gian giao dịch"].dt.time

# Chỉ giữ lại phần ngày ở cột Thời gian giao dịch
transaction_df["Thời gian giao dịch"] = transaction_df["Thời gian giao dịch"].dt.date

transaction_df

,Số CIF,Tên khách hàng,Dịch vụ,Tổng số tiền giao dịch,Đơn vị tiền tệ,Mã lỗi,Mô tả mã lỗi,Mã giao dịch,Thời gian giao dịch,Trạng thái,time
0,4000000118280,NGUYEN ANH TUAN,Chuyển tiền nhanh 247 qua tài khoản,2620000,VND,0,Hạch toán Thành công,00000FT542403206051,2024-02-01,Giao dịch thành công,23:38:53
1,3600200011641,BUI THI HUONG,Chuyển tiền nhanh 247 qua tài khoản,1490000,VND,0,Hạch toán Thành công,00000FT542403206052,2024-02-01,Giao dịch thành công,23:38:51
2,6000200061082,NGUYEN THI HONG HAU,Chuyển tiền nhanh 247 qua tài khoản,12000000,VND,98,Hạch toán Timeout,00000FT542403206049,2024-02-01,Giao dịch nghi vấn,23:38:28
3,7200000014564,TRAN THANH LOC,Chuyển tiền nhanh 247 qua tài khoản,100000,VND,0,Hạch toán Thành công,00000FT542403206047,2024-02-01,Giao dịch thành công,23:38:08
4,3100000041542,DANG THI LUYEN,Hóa đơn tiền điện,538428,VND,0,Gạch nợ Thành công,00000FT602403293681,2024-02-01,Giao dịch thành công,23:38:07
...,...,...,...,...,...,...,...,...,...,...,...
755013,6000000141882,NGUYEN HONG PHONG,Chuyển tiền nhanh 247 qua tài khoản,1000000,VND,0,Hạch toán Thành công,00000FT542406201987,2024-03-02,Giao dịch thành công,00:07:07
755014,6000000134751,DAU THI THAO,Chuyển tiền nhanh 247 qua tài khoản,18000000,VND,0,Hạch toán Thành công,00000FT542406201971,2024-03-02,Giao dịch thành công,00:06:43
755015,8900000034641,NGUYEN THI TUYET TRINH,Hóa đơn tiền điện,718430,VND,0,Gạch nợ Thành công,00000FT602406200522,2024-03-02,Giao dịch thành công,00:05:36
755016,8900000034641,NGUYEN THI TUYET TRINH,Hóa đơn tiền điện,662431,VND,0,Gạch nợ Thành công,00000FT602406200521,2024-03-02,Giao dịch thành công,00:05:00


In [40]:
# Các cột ngày cần xử lý
date_cols = ["Thời gian giao dịch"]

# Áp dụng chuẩn hóa
for col in date_cols:
    if col in transaction_df.columns:
        transaction_df[col] = transaction_df[col].apply(normalize_date)

# Kiểm tra kết quả
transaction_df[date_cols]


,Thời gian giao dịch
0,2024-02-01
1,2024-02-01
2,2024-02-01
3,2024-02-01
4,2024-02-01
...,...
755013,2024-03-02
755014,2024-03-02
755015,2024-03-02
755016,2024-03-02


In [41]:
# Thêm cột thời gian (sáng, trưa, chiều, tối vào phục vụ report)
# Hàm phân loại thời điểm trong ngày
def categorize_time(t):
    hour = t.hour
    if 5 <= hour < 12:
        return "Sáng"
    elif 12 <= hour < 14:
        return "Trưa"
    elif 14 <= hour < 18:
        return "Chiều"
    elif 18 <= hour < 23:
        return "Tối"
    else:
        return "Đêm"

# Tạo cột mới
transaction_df["Thời điểm trong ngày"] = transaction_df["time"].apply(categorize_time)
transaction_df

,Số CIF,Tên khách hàng,Dịch vụ,Tổng số tiền giao dịch,Đơn vị tiền tệ,Mã lỗi,Mô tả mã lỗi,Mã giao dịch,Thời gian giao dịch,Trạng thái,time,Thời điểm trong ngày
0,4000000118280,NGUYEN ANH TUAN,Chuyển tiền nhanh 247 qua tài khoản,2620000,VND,0,Hạch toán Thành công,00000FT542403206051,2024-02-01,Giao dịch thành công,23:38:53,Đêm
1,3600200011641,BUI THI HUONG,Chuyển tiền nhanh 247 qua tài khoản,1490000,VND,0,Hạch toán Thành công,00000FT542403206052,2024-02-01,Giao dịch thành công,23:38:51,Đêm
2,6000200061082,NGUYEN THI HONG HAU,Chuyển tiền nhanh 247 qua tài khoản,12000000,VND,98,Hạch toán Timeout,00000FT542403206049,2024-02-01,Giao dịch nghi vấn,23:38:28,Đêm
3,7200000014564,TRAN THANH LOC,Chuyển tiền nhanh 247 qua tài khoản,100000,VND,0,Hạch toán Thành công,00000FT542403206047,2024-02-01,Giao dịch thành công,23:38:08,Đêm
4,3100000041542,DANG THI LUYEN,Hóa đơn tiền điện,538428,VND,0,Gạch nợ Thành công,00000FT602403293681,2024-02-01,Giao dịch thành công,23:38:07,Đêm
...,...,...,...,...,...,...,...,...,...,...,...,...
755013,6000000141882,NGUYEN HONG PHONG,Chuyển tiền nhanh 247 qua tài khoản,1000000,VND,0,Hạch toán Thành công,00000FT542406201987,2024-03-02,Giao dịch thành công,00:07:07,Đêm
755014,6000000134751,DAU THI THAO,Chuyển tiền nhanh 247 qua tài khoản,18000000,VND,0,Hạch toán Thành công,00000FT542406201971,2024-03-02,Giao dịch thành công,00:06:43,Đêm
755015,8900000034641,NGUYEN THI TUYET TRINH,Hóa đơn tiền điện,718430,VND,0,Gạch nợ Thành công,00000FT602406200522,2024-03-02,Giao dịch thành công,00:05:36,Đêm
755016,8900000034641,NGUYEN THI TUYET TRINH,Hóa đơn tiền điện,662431,VND,0,Gạch nợ Thành công,00000FT602406200521,2024-03-02,Giao dịch thành công,00:05:00,Đêm


# in và load data dạng csv ra ngoài